In [1]:
import json
from typing import Union
import os
from fastapi import FastAPI, HTTPException
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from vanna.chromadb import ChromaDB_VectorStore
from vanna.openai import OpenAI_Chat
from chromadb.utils import embedding_functions
from openai import OpenAI
from pydantic import BaseModel

load_dotenv(find_dotenv())


True

In [2]:

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, client=None, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self,client=client, config=config)


In [3]:
# Select the embedding model to use.
# List of model names can be found here https://www.sbert.net/docs/pretrained_models.html
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="BAAI/bge-large-zh-v1.5",
)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
)
vn = MyVanna(
    client=client,
    config={
        "model": "qwen25:72b",
        "temperature": 0,
        "language": "chinese",
        "path": "..\chromadb_data",
        "embedding_function": sentence_transformer_ef,
    }
)

vn.connect_to_mysql(host=os.getenv("DORIS_HOST"), dbname=os.getenv("DORIS_DATABASE"), user=os.getenv("DORIS_USER"), password=os.getenv("DORIS_PASSWORD"), port=int(os.getenv("DORIS_PORT")))

e:\CodeTool\Anaconda3\envs\poetryenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
srm_pur_po_dll = '''
    -- sdhq.ods_v_srm_pur_po definition 采购订单表

CREATE TABLE `ods_v_srm_pur_po` (
  `item_id` BIGINT NOT NULL AUTO_INCREMENT(1) COMMENT '采购项目ID',
  `created_by_code` VARCHAR(255) NULL COMMENT '采购订单行创建人员工编号',
  `created_by_name` VARCHAR(255) NULL COMMENT '采购订单行创建人名称',
  `updated_by_code` VARCHAR(255) NULL COMMENT '采购订单行更新人员工编号',
  `updated_by_name` VARCHAR(255) NULL COMMENT '采购订单行更新人名称',
  `created_at` DATETIME NULL COMMENT '采购订单行创建时间',
  `updated_at` DATETIME NULL COMMENT '采购订单行更新时间',
  `item_deleted` TINYINT NULL COMMENT '采购订单行逻辑删除标识。非0为删除；0正常',
  `po_item_code` VARCHAR(255) NULL COMMENT '采购订单行编号',
  `po_item_qty_pur` DECIMAL(18, 2) NULL COMMENT '订单数量',
  `po_item_qty_ful` DECIMAL(18, 2) NULL COMMENT '累计入库数量',
  `price_gross` DECIMAL(18, 2) NULL COMMENT '含税单价',
  `amount_gross` DECIMAL(18, 2) NULL COMMENT '含税总金额',
  `amount_net` DECIMAL(18, 2) NULL COMMENT '不含税金额',
  `tax_amount` DECIMAL(18, 2) NULL COMMENT '税额',
  `mat_code` VARCHAR(255) NULL COMMENT '采购订单行 物料信息编码',
  `mat_name` VARCHAR(500) NULL COMMENT '采购订单行 物料信息名称',
  `ext_wq_specification` VARCHAR(500) NULL COMMENT '采购订单行 物料信息规格型号',
  `uom_desc` VARCHAR(255) NULL COMMENT '采购订单行 物料单位',
  `mat_cate_code` VARCHAR(255) NULL COMMENT '采购订单行 物料分类编码',
  `mat_cate_name` VARCHAR(255) NULL COMMENT '采购订单行 物料分类名称',
  `mat_prop` VARCHAR(255) NULL COMMENT '采购订单行 物料属性',
  `po_item_qty_rg` DECIMAL(18, 2) NULL COMMENT '累计申请退货数量',
  `pur_po_head_tr_id` BIGINT NULL COMMENT '采购订单表头id',
  `po_item_qty_dn` DECIMAL(18, 2) NULL COMMENT '在途数量',
  `arrival_quantity` DECIMAL(18, 2) NULL COMMENT '已到货未入库数量',
  `exchange_quantity` DECIMAL(18, 2) NULL COMMENT '换货数量',
  `item_status` VARCHAR(255) NULL COMMENT '采购订单行状态',
  `del_remain_qty` DECIMAL(18, 2) NULL COMMENT '剩余可发货数量',
  `head_id` BIGINT NULL COMMENT '采购订单头表id',
  `head_deleted` TINYINT NULL COMMENT '采购订请单头表逻辑删除标识。非0为删除；0正常',
  `po_code` VARCHAR(255) NULL COMMENT '采购订请单头表订单编号',
  `status` VARCHAR(255) NULL COMMENT '采购订请单头表状态',
  `ext_wq_sync_state` VARCHAR(255) NULL COMMENT '采购订请单头表同步状态',
  `ext_wq_buy_type` VARCHAR(255) NULL COMMENT '采购订请单头表请购类型',
  `ext_wq_po_type` VARCHAR(255) NULL COMMENT '采购订请单头表单据类型—魏桥',
  `bud_project_name` VARCHAR(255) NULL COMMENT '施工项目名称',
  `ext_inv_org_unit_name` VARCHAR(255) NULL COMMENT '需求部门名称',
  `pur_org_name` VARCHAR(255) NULL COMMENT '采购组织名称',
  `vend_name` VARCHAR(255) NULL COMMENT '供应商名称',
  `inv_org_name` VARCHAR(255) NULL COMMENT '库存组织名称',
  `industry_type` VARCHAR(255) NULL COMMENT '库存组织所属行业属性',
  `inv_loc_name` VARCHAR(255) NULL COMMENT '库存地点名称',
  `elt_time` DATETIME NULL COMMENT '抽取时间'
) ENGINE=OLAP
UNIQUE KEY(`item_id`)
COMMENT '采购订单视图表'
DISTRIBUTED BY HASH(`item_id`) BUCKETS AUTO
PROPERTIES (
"replication_allocation" = "tag.location.default: 1",
"min_load_replica_num" = "-1",
"is_being_synced" = "false",
"storage_medium" = "hdd",
"storage_format" = "V2",
"inverted_index_storage_format" = "V1",
"enable_unique_key_merge_on_write" = "true",
"light_schema_change" = "true",
"disable_auto_compaction" = "false",
"enable_single_replica_compaction" = "false",
"group_commit_interval_ms" = "10000",
"group_commit_data_bytes" = "134217728"
);
'''

In [5]:
srm_pur_pr_ddl = '''
-- sdhq.ods_v_srm_pur_pr definition  采购申请表

CREATE TABLE `ods_v_srm_pur_pr` (
  `item_id` BIGINT NOT NULL AUTO_INCREMENT(1) COMMENT '采购项目ID',
  `created_by_code` VARCHAR(255) NULL COMMENT '采购申请行创建人员工编号',
  `created_by_name` VARCHAR(255) NULL COMMENT '采购申请行创建人名称',
  `updated_by_code` VARCHAR(255) NULL COMMENT '采购申请行更新人员工编号',
  `updated_by_name` VARCHAR(255) NULL COMMENT '采购申请行更新人名称',
  `created_at` DATETIME NULL COMMENT '采购申请行创建时间',
  `updated_at` DATETIME NULL COMMENT '采购申请行更新时间',
  `item_deleted` TINYINT NULL COMMENT '采购申请行逻辑删除标识。非0为删除；0正常',
  `pr_item_status` VARCHAR(255) NULL COMMENT '采购申请行 行状态',
  `pr_qty` DOUBLE NULL COMMENT '采购申请行申请数量',
  `mat_code` VARCHAR(255) NULL COMMENT '采购申请行 物料信息编码',
  `mat_name` VARCHAR(600) NULL COMMENT '采购申请行 物料信息名称',
  `ext_wq_specification` VARCHAR(600) NULL COMMENT '采购申请行 物料信息规格型号',
  `uom_desc` VARCHAR(255) NULL COMMENT '采购申请行 物料单位',
  `mat_cate_code` VARCHAR(255) NULL COMMENT '采购申请行 物料分类编码',
  `mat_cate_name` VARCHAR(255) NULL COMMENT '采购申请行 物料分类名称',
  `mat_prop` VARCHAR(255) NULL COMMENT '采购申请行 物料属性',
  `pur_employee_code` VARCHAR(255) NULL COMMENT '采购员编码',
  `pur_employee_name` VARCHAR(255) NULL COMMENT '采购员名称',
  `pur_pr_head_tr_id` BIGINT NULL COMMENT '采购申请单据表id',
  `ext_wq_sync_status` VARCHAR(255) NULL COMMENT '采购申请行同步状态',
  `head_id` BIGINT NULL COMMENT '采购申请单id',
  `head_deleted` TINYINT NULL COMMENT '采购申请单逻辑删除标识。非0为删除；0正常',
  `pr_code` VARCHAR(255) NULL COMMENT '采购申请单编号',
  `pr_status` VARCHAR(255) NULL COMMENT '采购申请单状态',
  `approve_status` VARCHAR(255) NULL COMMENT '审批状态',
  `origin` VARCHAR(255) NULL COMMENT '来源',
  `stock_control_name` VARCHAR(255) NULL COMMENT '库存组织名称',
  `industry_type` VARCHAR(255) NULL COMMENT '库存组织所属行业类型',
  `buy_type` VARCHAR(255) NULL COMMENT '请购类型',
  `pur_org_name` VARCHAR(255) NULL COMMENT '采购组织名称',
  `sync_status` VARCHAR(255) NULL COMMENT '同步状态',
  `pr_date_apply` DATE NULL COMMENT '申请日期',
  `buy_store_name` VARCHAR(255) NULL COMMENT '需求仓库名称（需求公司名称）',
  `buy_department_name` VARCHAR(255) NULL COMMENT '需求部门名称',
  `ext_wq_erp_code` VARCHAR(255) NULL COMMENT '外部单据编号',
  `use_to` VARCHAR(255) NULL COMMENT '用途',
  `plan_type_name` VARCHAR(255) NULL COMMENT '计划类型名称',
  `ext_wq_task_name` VARCHAR(255) NULL COMMENT '施工项目名称',
  `elt_time` DATETIME NULL COMMENT '抽取时间'
) ENGINE=OLAP
UNIQUE KEY(`item_id`)
COMMENT '采购申请视图表'
DISTRIBUTED BY HASH(`item_id`) BUCKETS AUTO
PROPERTIES (
"replication_allocation" = "tag.location.default: 1",
"min_load_replica_num" = "-1",
"is_being_synced" = "false",
"storage_medium" = "hdd",
"storage_format" = "V2",
"inverted_index_storage_format" = "V1",
"enable_unique_key_merge_on_write" = "true",
"light_schema_change" = "true",
"disable_auto_compaction" = "false",
"enable_single_replica_compaction" = "false",
"group_commit_interval_ms" = "10000",
"group_commit_data_bytes" = "134217728"
);
'''

In [6]:
vn.train(ddl=srm_pur_po_dll)
vn.train(ddl=srm_pur_pr_ddl)

Adding ddl: 
    -- sdhq.ods_v_srm_pur_po definition 采购订单表

CREATE TABLE `ods_v_srm_pur_po` (
  `item_id` BIGINT NOT NULL AUTO_INCREMENT(1) COMMENT '采购项目ID',
  `created_by_code` VARCHAR(255) NULL COMMENT '采购订单行创建人员工编号',
  `created_by_name` VARCHAR(255) NULL COMMENT '采购订单行创建人名称',
  `updated_by_code` VARCHAR(255) NULL COMMENT '采购订单行更新人员工编号',
  `updated_by_name` VARCHAR(255) NULL COMMENT '采购订单行更新人名称',
  `created_at` DATETIME NULL COMMENT '采购订单行创建时间',
  `updated_at` DATETIME NULL COMMENT '采购订单行更新时间',
  `item_deleted` TINYINT NULL COMMENT '采购订单行逻辑删除标识。非0为删除；0正常',
  `po_item_code` VARCHAR(255) NULL COMMENT '采购订单行编号',
  `po_item_qty_pur` DECIMAL(18, 2) NULL COMMENT '订单数量',
  `po_item_qty_ful` DECIMAL(18, 2) NULL COMMENT '累计入库数量',
  `price_gross` DECIMAL(18, 2) NULL COMMENT '含税单价',
  `amount_gross` DECIMAL(18, 2) NULL COMMENT '含税总金额',
  `amount_net` DECIMAL(18, 2) NULL COMMENT '不含税金额',
  `tax_amount` DECIMAL(18, 2) NULL COMMENT '税额',
  `mat_code` VARCHAR(255) NULL COMMENT '采购订单行 物料信息编码',
  `ma

Insert of existing embedding ID: 457a612f-e13a-5ff5-8e41-1ea1dc4f4564-ddl
Add of existing embedding ID: 457a612f-e13a-5ff5-8e41-1ea1dc4f4564-ddl


Adding ddl: 
-- sdhq.ods_v_srm_pur_pr definition  采购申请表

CREATE TABLE `ods_v_srm_pur_pr` (
  `item_id` BIGINT NOT NULL AUTO_INCREMENT(1) COMMENT '采购项目ID',
  `created_by_code` VARCHAR(255) NULL COMMENT '采购申请行创建人员工编号',
  `created_by_name` VARCHAR(255) NULL COMMENT '采购申请行创建人名称',
  `updated_by_code` VARCHAR(255) NULL COMMENT '采购申请行更新人员工编号',
  `updated_by_name` VARCHAR(255) NULL COMMENT '采购申请行更新人名称',
  `created_at` DATETIME NULL COMMENT '采购申请行创建时间',
  `updated_at` DATETIME NULL COMMENT '采购申请行更新时间',
  `item_deleted` TINYINT NULL COMMENT '采购申请行逻辑删除标识。非0为删除；0正常',
  `pr_item_status` VARCHAR(255) NULL COMMENT '采购申请行 行状态',
  `pr_qty` DOUBLE NULL COMMENT '采购申请行申请数量',
  `mat_code` VARCHAR(255) NULL COMMENT '采购申请行 物料信息编码',
  `mat_name` VARCHAR(600) NULL COMMENT '采购申请行 物料信息名称',
  `ext_wq_specification` VARCHAR(600) NULL COMMENT '采购申请行 物料信息规格型号',
  `uom_desc` VARCHAR(255) NULL COMMENT '采购申请行 物料单位',
  `mat_cate_code` VARCHAR(255) NULL COMMENT '采购申请行 物料分类编码',
  `mat_cate_name` VARCHAR(255) NULL COMMENT 

Insert of existing embedding ID: fe0f0061-1523-5f87-ad89-210accf3ee52-ddl
Add of existing embedding ID: fe0f0061-1523-5f87-ad89-210accf3ee52-ddl


'fe0f0061-1523-5f87-ad89-210accf3ee52-ddl'

In [7]:
question = '订单数量排名前五的采购订单信息,查询的字段必须有其对应的中文别名。'
sql_query = vn.generate_sql(question)

Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2


SQL Prompt: [{'role': 'system', 'content': 'You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    -- sdhq.ods_v_srm_pur_po definition 采购订单表\n\nCREATE TABLE `ods_v_srm_pur_po` (\n  `item_id` BIGINT NOT NULL AUTO_INCREMENT(1) COMMENT \'采购项目ID\',\n  `created_by_code` VARCHAR(255) NULL COMMENT \'采购订单行创建人员工编号\',\n  `created_by_name` VARCHAR(255) NULL COMMENT \'采购订单行创建人名称\',\n  `updated_by_code` VARCHAR(255) NULL COMMENT \'采购订单行更新人员工编号\',\n  `updated_by_name` VARCHAR(255) NULL COMMENT \'采购订单行更新人名称\',\n  `created_at` DATETIME NULL COMMENT \'采购订单行创建时间\',\n  `updated_at` DATETIME NULL COMMENT \'采购订单行更新时间\',\n  `item_deleted` TINYINT NULL COMMENT \'采购订单行逻辑删除标识。非0为删除；0正常\',\n  `po_item_code` VARCHAR(255) NULL COMMENT \'采购订单行编号\',\n  `po_item_qty_pur` DECIMAL(18, 2) NULL COMMENT \'订单数量\',\n  `po_item_qty_ful` DECIMAL(18, 2) NULL COMMENT

In [8]:
df_result = vn.run_sql(sql_query)
df_result

,采购项目ID,采购订单行创建人员工编号,采购订单行创建人名称,采购订单行更新人员工编号,采购订单行更新人名称,采购订单行创建时间,采购订单行更新时间,采购订单行逻辑删除标识,采购订单行编号,订单数量,...,采购订请单头表请购类型,采购订请单头表单据类型—魏桥,施工项目名称,需求部门名称,采购组织名称,供应商名称,库存组织名称,库存组织所属行业属性,库存地点名称,抽取时间
0,2293413,1700435,刘及震,1282386,成鹏飞62825,2024-07-12 16:49:07,2024-07-23 11:48:10,0,POI20240712100805,158580000.00,...,设备物资,常规采购订单,None,魏桥砚山新能源有限公司项目部,集团板块采购部,宁波欧达光电有限公司,魏桥砚山新能源有限公司,新能源,魏桥砚山新能源固定资产仓库,2024-09-13 07:58:55
1,2769208,1846160,高海伦,1846160,高海伦,2024-12-03 17:28:58,2024-12-03 17:29:21,0,POI20241203100774,7000000.00,...,ZR01原辅料采购申请,常规采购订单,None,None,宏拓板块采购部,南京云海铝业有限公司,邹平宏发铝业科技有限公司,氧化铝,熔铸原料仓,2024-12-04 02:01:16
2,2612369,1333614,张少英62975,LAIGANGYG,LAIGANGYG,2024-10-21 15:54:22,2024-10-21 19:44:49,0,POI20241021101174,6119125.00,...,基建材料,常规采购订单,None,None,宏拓板块采购部,莱钢集团烟台钢管有限公司,滨州市沾化区汇宏新材料有限公司,氧化铝,沾化汇宏粉煤灰基建材料仓库（3）,2024-10-22 02:00:29
3,2610004,1550942,崔全梅,None,None,2024-10-18 07:59:27,2024-10-18 10:02:34,0,POI20241018100033,5595092.00,...,原材料,常规采购订单,None,None,宏拓板块采购部,山东铁雄冶金科技有限公司,邹平县汇茂新材料科技有限公司,氧化铝,邹平汇茂西区一厂原材料仓库,2024-10-19 02:00:28
4,2552124,1550942,崔全梅,None,None,2024-10-02 09:50:31,2024-10-02 10:01:30,0,POI20241002100187,5521155.00,...,原材料,常规采购订单,None,None,宏拓板块采购部,山东铁雄冶金科技有限公司,邹平县汇茂新材料科技有限公司,氧化铝,邹平汇茂西区一厂原材料仓库,2024-10-03 02:00:23


In [9]:
poltly_code  = vn.generate_plotly_code(question,sql_query,df_result)
poltly_code

Using model qwen25:72b for 981.75 tokens (approx)


'import plotly.express as px\nimport plotly.graph_objects as go\n\nif len(df) == 1:\n    fig = go.Figure(go.Indicator(\n        mode = "number",\n        value = df[\'订单数量\'].iloc[0],\n        title = {"text": "订单数量"}\n    ))\nelse:\n    fig = px.bar(df, x=\'采购订单行编号\', y=\'订单数量\', title=\'订单数量排名前五的采购订单信息\')\n\n'

In [10]:
figure = vn.get_plotly_figure(poltly_code,df_result,dark_mode=False)
figure

In [11]:
import plotly.io as pio
plotly_json = pio.to_json(figure)



In [12]:
sumamary  = vn.generate_summary(question,df_result)
sumamary

Using model qwen25:72b for 1621.0 tokens (approx)


'根据订单数量排名前五的采购订单信息如下：\n\n1. 采购项目ID：2293413，订单数量：158,580,000，物料信息名称：光伏组件_详见技术协议，供应商名称：宁波欧达光电有限公司。\n2. 采购项目ID：2769208，订单数量：7,000,000，物料信息名称：铝锰中间合金_ Al-Mn10，供应商名称：南京云海铝业有限公司。\n3. 采购项目ID：2612369，订单数量：6,119,120，物料信息名称：无缝钢管_Φ219*12 Q345B 定尺10810mm GB6479-2013，供应商名称：莱钢集团烟台钢管有限公司。\n4. 采购项目ID：2610004，订单数量：5,595,090，物料信息名称：煤气_焦化气，供应商名称：山东铁雄冶金科技有限公司。\n5. 采购项目ID：2552124，订单数量：5,521,160，物料信息名称：煤气_焦化气，供应商名称：山东铁雄冶金科技有限公司。'

In [13]:
from plotly.io import from_json

fig = from_json(plotly_json)
fig.show()